In [1]:
from datetime import datetime
from hashlib import sha256
import pandas as pd
import random
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Create a dataframe with name, birthday
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Dave'],
    'Birthday': ['2005-05-20', '1990-02-15', '2008-11-10', '2008-11-10']
}

# Create a DataFrame
df = pd.DataFrame(data)

# Function to calculate age from birthday
def calculate_age(birthdate):
    birthdate = datetime.strptime(birthdate, '%Y-%m-%d')
    today = datetime.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

# Function to generate a group signature key pair
def generate_group_signature_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key

# Function to sign age with group signature
def sign_age_with_group_signature(private_key, age):
    signature = private_key.sign(
        str(age).encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

# Function to verify age signature with group public key
def verify_age_signature_with_group_public_key(public_key, age, signature):
    try:
        public_key.verify(
            signature,
            str(age).encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False

# Function to generate ZKP proof for age over 18
def generate_zkp_proof_age_over_18(age):
    # In a real implementation, this function would generate a ZKP proof
    # Here we simulate it by hashing the age and returning the hash
    return sha256(str(age).encode()).hexdigest()

# Function to verify ZKP proof for age over 18
def verify_zkp_proof_age_over_18(zkp_proof, age):
    # In a real implementation, this function would verify the ZKP proof
    # Here we simulate it by hashing the age and comparing it to the provided proof
    return sha256(str(age).encode()).hexdigest() == zkp_proof

# Generate group signature key pair
group_private_key, group_public_key = generate_group_signature_key_pair()

# Create empty lists to store validation results
validations = []

# Iterate over each row of the dataframe
for index, row in df.iterrows():
    age = calculate_age(row['Birthday'])
    is_over_18 = age >= 18
    

    if is_over_18:
        # Generate ZKP proof for age over 18
        zkp_proof = generate_zkp_proof_age_over_18(age)

        # Sign age with group signature
        signature = sign_age_with_group_signature(group_private_key, age)

        # Verify ZKP proof and group signature
        zkp_proof_valid = verify_zkp_proof_age_over_18(zkp_proof, age)
        group_signature_valid = verify_age_signature_with_group_public_key(group_public_key, age, signature)

        # Add validation results to the dataframe
        validations.append((row['Name'], zkp_proof_valid, group_signature_valid))
    else:
        # For individuals under 18, no need for ZKP proof or group signature
        validations.append((row['Name'], False, False))

# Add validation results to the dataframe
validation_data = {
    'Name': [validation[0] for validation in validations],
    'ZKP Proof Valid': [validation[1] for validation in validations],
    'Group Signature Valid': [validation[2] for validation in validations]
}
validation_df = pd.DataFrame(validation_data)

# Print the new DataFrame
print("\nValidation Results DataFrame:")
print(validation_df)



Validation Results DataFrame:
      Name  ZKP Proof Valid  Group Signature Valid
0    Alice             True                   True
1      Bob             True                   True
2  Charlie            False                  False
3     Dave            False                  False
